<a href="https://colab.research.google.com/github/zeinabmohamed/CIT690E-DeepLearning-Zeinab_Abdelmawla-191009/blob/main/Chatbot/MaLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm


In [ ]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
!pip install --user -U nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
# from keras.optimizers import Adadelta
from tensorflow import keras
from tensorflow.keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download quora-question-pairs -f data?select=test.csv.zip



In [ ]:
! kaggle competitions download quora-question-pairs -f test.csv.zip
! kaggle competitions download quora-question-pairs -f train.csv.zip

! unzip -o test.csv.zip
! unzip -o train.csv.zip

In [ ]:
!pip install wget

import wget
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = wget.download(url)

In [ ]:
# File paths
#https://www.kaggle.com/c/quora-question-pairs/data?select=test.csv.zip
TRAIN_CSV = './train.csv'
TEST_CSV = './test.csv'
EMBEDDING_FILE = './GoogleNews-vectors-negative300.bin.gz'
MODEL_SAVING_DIR = '/home/ecohen/HDD/HDD4/Models/Kaggle/Quora/'

In [ ]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [ ]:
type(train_df)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stops = set(stopwords.words('english'))
stops

In [ ]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text


In [ ]:
  # Example 
  sample = "I am Zeinab"
  sample = re.sub(r"Zeinab", "i", sample)
  sample

In [ ]:
# Example 
text_to_word_list("How do I out get rid of n't Erectile Dysfunction?")

In [ ]:
#Example
#word2vec.vocab['egypt']
#'egypt' not in word2vec.vocab
#word2vec.vocab['of']
'of' not in word2vec.vocab

In [ ]:
# Prepare embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
#Example
len(word2vec['egypt'])

In [ ]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding

In [ ]:
dataset = pd.DataFrame()
# Iterate through the text of both questions of the row
#for question in questions_cols:
vocabulary = dict()
inverse_vocabulary = ['<unk>']
q2n = []  # q2n -> question numbers representation
for word in text_to_word_list("How zeinab shouldn do I out get rid of n't zeinab Erectile Dysfunction?"):
  # Check for unwanted words
  if word in stops and word not in word2vec.vocab:
      print('Stop word : ',word)
      continue

  if word not in vocabulary:
      print('Other word : ',word)
      vocabulary[word] = len(inverse_vocabulary)
      q2n.append(len(inverse_vocabulary))
      inverse_vocabulary.append(word)
  else:
      print('else word : ',word)
      q2n.append(vocabulary[word])

    # Replace questions as word to question as number representation
    # print(0)
    # print(question)
    # print(q2n)
print(vocabulary)
print(q2n)

In [ ]:
# Example
word2vec['egypt']
similarity = word2vec.similarity('egypt', 'car')
similarity

In [ ]:
from tqdm import tqdm

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
questions_cols = ['question1', 'question2']
# Iterate over the questions only of both training and test datasets
for dataset in tqdm([train_df, test_df]):
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])
            # Replace questions as word to question as number representation
            dataset.at[index, question]=q2n            

In [ ]:
dataset

In [ ]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [ ]:
embeddings.shape

In [ ]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max(),
                     test_df.question1.map(lambda x: len(x)).max(),
                     test_df.question2.map(lambda x: len(x)).max())

max_seq_length

In [ ]:
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

In [ ]:
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

In [ ]:
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

In [ ]:
 embedding_layer.build(())

In [ ]:
embedding_layer.weights

In [ ]:
#word2vec[word2vec.index2word[6]]